# `app.ipynb`

* Creates a class that can make API requests and store them for later use.
* Does not provide user interface elements, except persistant class variables (index, columns) are stored as widgets.  Widgets provides both a programmatic and non-programmatic interface to explore data.

In [66]:
import traitlets, requests, ipywidgets, pandas, IPython, odo, multipledispatch, blaze, logging, http, sys, json
get_ipython = ipywidgets.get_ipython
%reload_ext autoreload
%autoreload 2

In [64]:
@odo.resource.register( 'gh://.*' )
def github__shorthand_resource( path, *args, **kwargs ):
    """Get user or repository level information."""
    __route__ = 'gh://'
    user = path.split( __route__ )[1].split('/')
    user, repo = user if len(user) == 2 else [user[0],'']
    method = 'repos' if repo else 'users'    
    return 'https://api.github.com/{method}/{user}/{repo}'.format( method=method, user=user,repo=repo)

In [3]:
@multipledispatch.dispatch( str, dict, requests.sessions.Session )
def github_request( url, params = {}, session = requests.Session() ):
    """
    Make the URL when a url is passed to the function
    
            req, resp = github_request('tonyfast', {}, requests.Session() )
    """
    if not url.startswith('http'):
        url = odo.resource( 'gh://' + url )
    request = requests.Request('GET', url=url, params=params.copy(), headers={'User-Agent': 'tonyfast'} )
    request.params =  request.params 
    return request, session.send( request.prepare() )

@multipledispatch.dispatch( requests.models.Request, requests.models.Response )
def github_request( requests, response ):
    del requests.params['access_token']
    return requests.prepare(), response.json()

In [1]:
class Projects( traitlets.HasTraits ):
    current = traitlets.List([])
    access_token = ipywidgets.Text('asdfasdf', description='Github Access Token')
    username = ipywidgets.Text('tonyfast', description='Github Username')
    df = pandas.DataFrame()
    session = requests.Session()  # Eventually get information from the logger
    catalog = {}   # An object of native python classes to manipulate the Projects
    
    def __init__( self, projects = [], *args, **kwargs ):
        for project in projects:
            self.add_project( project )
        
    @traitlets.observe('current')
    def _update_projects(self, change, *args, **kwargs):
        for project in change['new']:            
            if not project in change['old']:
                self._make_request( project, alias=project )

    def _make_request( self, project_or_url, params = {}, alias=None ):  
        params['access_token'] = self.access_token.value
        recorded = self._record( *github_request( project_or_url, params, self.session ) )
        if alias:
            if isinstance( alias, str ):
                self.catalog[alias] = recorded
        
    def _record( self, request, response ):
        del request.params['access_token']
        url = request.prepare().url
        if isinstance( url, str ):
            self.catalog[url] = {
                'response': response,
                'time': str(pandas.datetime.now()),
                'params': request.params.copy(),
                'request': request,
                'url': url
            } 
            obj = self.catalog[url]['response'].json()
            self._update_dataframe( obj )
            return self.catalog[url] 

    def _update_dataframe( self, obj ):
        if isinstance( obj, list ):
            self.df = self.df.append( obj )
            
    def _paginate( self, project_or_url, params ):
        params['page'] = 0
        for k,v in self.catalog.items(): 
            if k.startswith(project_or_url) and isinstance( v['response'].json(), list ):
                if 'page' in v['params']:
                    params['page'] = max( params['page'], v['params']['page'])
        params['page'] += 1
        return params
                                
    def more( self, project_or_url, params = {} ):
        params = self._paginate( project_or_url, params )
        self._make_request( project_or_url, params, self.session )
        
    def add_project( self, project ):
        tmp = self.current.copy()
        tmp.append(project)
        self.current = tmp

NameError: name 'traitlets' is not defined

In [47]:
#projects = Projects(['bokeh/bokeh','jupyter/jupyter'])

In [65]:
#projects.more(projects.catalog['bokeh/bokeh']['response'].json()['issues_url'].split('{')[0])
#projects.more(projects.catalog['bokeh/bokeh']['response'].json()['issues_url'].split('{')[0])